In [31]:
import random
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import re
import torch
import ast

import albumentations as A

from torch.utils.data import Dataset, DataLoader

from PIL import Image

df = pd.read_csv("ODIR-5K_Training_Preprocessed.csv")

df = df[['ID', 'labels']]
df['labels'] = df['labels'].apply(lambda x: ast.literal_eval(x))
dfdummy = pd.get_dummies(df['labels'].apply(pd.Series).stack()).sum(level=0)
df = pd.concat([df, dfdummy], axis=1)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().


In [32]:
# Change every item in labels to classes from 0 to 7
df['labels'] = df['labels'].apply(lambda x: [0 if i == 'A' else 1 if i == 'C' else 2 if i == 'D' else 3 if i == 'G' else 4 if i == 'H' else 5 if i == 'M' else 6 if i == 'N' else 7 for i in x])

# Convert the labels from A, C,D, G, H, M, n and O to a column of lists
df['coded labels'] = df.apply(lambda x: [x['A'], x['C'], x['D'], x['G'], x['H'], x['M'], x['N'], x['O']], axis=1)

df['Name'] = df['ID'].apply(lambda x: os.path.splitext(x)[0])
df['Name'] = df['Name'].str.cat(df['coded labels'].astype(str), sep ="_")

df

,ID,labels,A,C,D,G,H,M,N,O,coded labels,Name
0,0_left.jpg,[1],0,1,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0]","0_left_[0, 1, 0, 0, 0, 0, 0, 0]"
1,0_right.jpg,[6],0,0,0,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 1, 0]","0_right_[0, 0, 0, 0, 0, 0, 1, 0]"
2,1_left.jpg,[6],0,0,0,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 1, 0]","1_left_[0, 0, 0, 0, 0, 0, 1, 0]"
3,1_right.jpg,[6],0,0,0,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 1, 0]","1_right_[0, 0, 0, 0, 0, 0, 1, 0]"
4,2_left.jpg,"[2, 7]",0,0,1,0,0,0,0,1,"[0, 0, 1, 0, 0, 0, 0, 1]","2_left_[0, 0, 1, 0, 0, 0, 0, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...
6563,4689_right.jpg,[6],0,0,0,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 1, 0]","4689_right_[0, 0, 0, 0, 0, 0, 1, 0]"
6564,4690_left.jpg,[2],0,0,1,0,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0, 0]","4690_left_[0, 0, 1, 0, 0, 0, 0, 0]"
6565,4690_right.jpg,[2],0,0,1,0,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0, 0]","4690_right_[0, 0, 1, 0, 0, 0, 0, 0]"
6566,4784_left.jpg,"[0, 4]",1,0,0,0,1,0,0,0,"[1, 0, 0, 0, 1, 0, 0, 0]","4784_left_[1, 0, 0, 0, 1, 0, 0, 0]"


In [33]:
output_path = "./squared_and_cropped_dataset/"

for item in range(len(df['ID'])):
    os.rename(output_path + df['ID'][item], output_path + df['Name'][item] + '.jpg')

train_img = [os.path.join(output_path, i) for i in os.listdir(output_path)]

['./squared_and_cropped_dataset/2965_left_[0, 0, 0, 0, 0, 0, 1, 0].jpg',
 './squared_and_cropped_dataset/4383_right_[0, 0, 1, 0, 0, 0, 0, 0].jpg',
 './squared_and_cropped_dataset/3194_left_[0, 0, 0, 0, 0, 0, 1, 0].jpg',
 './squared_and_cropped_dataset/4651_left_[0, 0, 1, 0, 0, 0, 0, 0].jpg',
 './squared_and_cropped_dataset/3135_left_[0, 0, 0, 0, 0, 0, 1, 0].jpg',
 './squared_and_cropped_dataset/2576_left_[0, 0, 0, 0, 0, 0, 1, 0].jpg',
 './squared_and_cropped_dataset/2627_right_[0, 0, 0, 0, 0, 0, 1, 0].jpg',
 './squared_and_cropped_dataset/2193_right_[0, 1, 0, 0, 0, 0, 0, 0].jpg',
 './squared_and_cropped_dataset/4676_right_[0, 0, 1, 0, 0, 0, 0, 0].jpg',
 './squared_and_cropped_dataset/2668_left_[0, 0, 0, 0, 0, 0, 1, 0].jpg',
 './squared_and_cropped_dataset/22_left_[0, 0, 1, 0, 0, 0, 0, 1].jpg',
 './squared_and_cropped_dataset/1617_left_[0, 0, 0, 0, 0, 1, 0, 0].jpg',
 './squared_and_cropped_dataset/3127_right_[0, 0, 0, 0, 0, 0, 1, 0].jpg',
 './squared_and_cropped_dataset/3157_left_[0, 0,

In [53]:

def createxy(image_list):
    X = []  # images
    y = []  # labels (0 for Normal or 1 for Pneumonia)
    pattern = r'\[([\d,\s]+)\]'

    for image in tqdm(image_list):
        try:
            img = cv2.imread(image)
            #img = img.astype(np.float32) / 255.
            X.append(img)
        except:
            continue

        match = re.search(pattern, image)
        extracted_list = match.group(1).split(',')
        extracted_list = [int(i) for i in extracted_list]
        y.append(extracted_list)

    return X, y
    
X_train, y_train = createxy(train_img)

100%|██████████| 6568/6568 [00:03<00:00, 1646.56it/s]


In [35]:
X_train = np.asarray(X_train, dtype=np.float32)
y_train = np.asarray(y_train, dtype=np.float32)

In [41]:
X_train[0]

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]], dtype=float32)

In [56]:

class ImageDataset(Dataset):
    def __init__(self, images, labels, transforms):
        self.X = images
        self.y = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        transformed = self.transforms(image=self.X[index])
        transformed_image = transformed["image"]
        transformed_image = transformed_image.transpose((2, 0, 1))
        target = self.y[index]
        target = np.expand_dims(target, axis=0)
        target = torch.from_numpy(target).float()

        return transformed_image, target

augmentations = A.Compose([
        A.Rotate(limit=15, p=0.5),  # Random rotation up to 15 degrees
        A.HorizontalFlip(p=0.5),   # Horizontal flipping with a 50% probability
        A.RandomBrightnessContrast(p=0.5),  # Random brightness and contrast adjustments
        A.ElasticTransform(alpha=1, sigma=10, alpha_affine=10, p=0.5),  # Elastic deformation
        A.Blur(blur_limit=(3, 7), p=0.5),  # Gaussian blur
        A.RandomGamma(p=0.5),  # Random gamma adjustment
        A.HueSaturationValue(p=0.5),  # Hue, saturation, and value adjustments
    ], p=1)

augmented_dataset = ImageDataset(X_train, y_train, augmentations)
augmented_dataloader = DataLoader(augmented_dataset, batch_size=1, shuffle=True)

In [30]:
#fix class imbalance

6568

In [ ]:
#split into train and validation sets